In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt 
import pandas as pd               # Python data science library
#import numpy as np
#import re
#import sys

from modelclass import model
#import modelpattern as pt
#import modelmanipulation as mp    # Module for model text processing
#from modelmanipulation import explode


# ModelFlow, A library to manage and solve Models
Sandbjerg november 2021

Ib Hansen <br> 
Ib.hansen.iv@gmail.com

Work done at Danmarks Nationalbank, ECB and Hansen Ølkonometri. 

**Problem**:<br>
Stress-test model for banks<br>
Complicated and slow **Excel** workbook <br>
Difficult to maintain and change

**Solution**:<br>
Separate specification (at a high level of abstraction) and solution code.<br> 
Python comes **batteries included**. Data management, text processing, visualization ...

**Implementation** of **minimum workable toolkit** <br>
A **transpiler**: Takes a model in a **domain specific** Business logic language:<br>
Create **solver** and **utility** functions using Python libraries. <br>


**Refactor and refine**<br>
Larger models, Faster transpiler, Newton and Gauss solvers, Logical structure, Derivatives, Visualization, Front ends ...


## Refine and refactor
To suit the needs of the different models thrown at the toolkit.

**Specify very large (or small) models** as concise and intuitive equations. 1 million equation and more can be handled.

**Agile model development** Model are specified at a high level of abstraction and are processed fast. Experiments with model specification are agile and fast. 

**Onboarding models and combining from different sources**. Recycling and combining models specified in different ways: Excel, Latex, Dynare, Python or other languages. 

**A rich set of analytical tools for model and result analytic** helps to understand the model and its results. 

**The user can extend and modify the tools** to her or his needs. All code is in Python and the core is quite small.       

## What is a Model in ModelFlow
ModelFlow is created to handle models. The term [**model**](https://en.wikipedia.org/wiki/Model) can mean a lot of different concepts. 

The scope of models handled by ModelFlow is **discrete** models which is the same for each time frame, can be formulated as **mathematical equations** and *can* have **lagged** and **leaded** variables. <br>
This allows the system to handle quite a large range of models.

A model with:

 - $\textbf n$ number of endogeneous variables
 - $\textbf k$ number of exogeneous variables 
 - $\textbf u$ max lead of endogeneous variables
 - $\textbf r$ max lag of endogeneous variables 
 - $\textbf s$ max lag of exogeneous variables 
 - $t$ time frame (year, quarter, day second or another another unit)
 
can be written in two ways, **normalized** or **un-normalized** form

### Normalized model
Each endogenous variable is on the left hand side one time - and only one time.<br>
\begin{eqnarray}
y_t^1 & = & f^1(y_{t+u}^1...,y_{t+u}^n...,y_t^2...,y_{t}^n...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^k,...x_{t-s}^1...,x_{t-s}^k) \\
y_t^2 & = & f^2(y_{t+u}^1...,y_{t+u}^n...,y_t^1...,y_{t}^n...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^k,...x_{t-s}^1...,x_{t-s}^k) \\
\vdots \\
y_t^n & = & f^n(y_{t+u}^1...,y_{t+u}^n...,y_t^1...,y_{t}^{n-1}...y_{t-r}^1...,y_{t-r}^n,x_t^1...x_{t}^r,x..._{t-s}^1...,x_{t-s}^k)
\end{eqnarray}


Written in matrix notation where  $\textbf{y}_t$ and $\textbf{x}_t$ are vectors of endogenous/exogenous  variables for time t<br>

\begin{eqnarray}  
\textbf{y}_t & = & \textbf{F}(\textbf{y}_{t+u} \cdots \textbf{y}_t \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-s})
\end{eqnarray}


ModelFlow allows variable (the  𝐱 'es and the  𝐲 'es the to be scalars, matrices, arrays or pandas dataframes.

### Un-normalized form
Some models can not easy be specified as normalized formulas. Especially   models with equilibrium  conditions can more suitable be specified in the more generalized un-normalized form.

Written in matrix notation like before:

\begin{eqnarray}  
\textbf{0}& = & \textbf{F}(\textbf{y}_{t+u} \cdots \textbf{y}_t \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-s})
\end{eqnarray}

The number of endogenous variables and equations should still be the same.

#### Model solution 
For a normalized model:

\begin{eqnarray}  
\textbf{y}_t & = & \textbf{F}(\textbf{y}_{t+u} \cdots \textbf{y}_t \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-r})     
\end{eqnarray}

a solution is  $\textbf{y}_t^*$ so that:


\begin{eqnarray}  
\textbf{y}_t^* & = & \textbf{F}(\textbf{y}_{t+u} \cdots \textbf{y}_t^* \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-r})     
\end{eqnarray}

For the un-normalized model:
\begin{eqnarray}  
\textbf{0}& = & \textbf{F}(\textbf{y}_{t+u} \cdots \textbf{y}_t \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-s})
\end{eqnarray}


a solution $\textbf{y}_t^*$ is 

\begin{eqnarray}  
\textbf{0} & = & \textbf{G}(\textbf{y}_{t+u} \cdots \textbf{y}_t^* \cdots \textbf{y}_{t-r},\textbf{x}_t \cdots \textbf{x}_{t-r})     
\end{eqnarray}

Some models can have more than one solution. In this case the solution can depend on the starting point of the solution algorithm. 

## Model derivatives 

Both for solving and for analyzing the causal structure of a model it can be useful to define different matrices of derivatives for a model $\textbf F()$ like this:

\begin{align}  
\textbf{A}_t & = & \frac{\partial \textbf{F}}{\partial \textbf{y}_t^T}  \hphantom{\hspace{5 mm} i=1, \cdots , r}   
 &\hspace{1 mm}\mbox{Derivatives with respect to current endogeneous variables} \\ \\
\textbf{E}_t^i & = & \frac{\partial \textbf{F}}{\partial \textbf{y}_{t-i}^T } \hspace{5 mm} i=1, \cdots , r  &\hspace{1 mm}\mbox{  Derivatives with respect to lagged endogeneous variables  } \\  \\
\textbf{D}_t^j & = & \frac{\partial \textbf{F}}{\partial \textbf{y}_{t+j}^T } \hspace{5 mm} j=1, \cdots , u  &\hspace{1 mm}\mbox{  Derivatives with respect to leaded endogeneous variables  } \\  \\
\textbf{F}_t^k & = & \frac{\partial \textbf{F}}{\partial \textbf{x}_{t-i} ^T} \hspace{5 mm} k=0, \cdots , s  &\hspace{1 mm}\mbox{  Derivatives with respect to current and lagged exogeneous variables  }\\ 
\end{align}

For un-normalized models the derivative matrices are just the dervatives of $\textbf G$ instead of $\textbf F$

## Model solutions

There are numerous methods to solve models (systems) as mentioned above. ModelFlow can apply 3 different types of model solution methods: 

 1. If the model has **no contemporaneous feedback**, the equations can be sorted 
 [Topological](https://en.wikipedia.org/wiki/Topological_sorting) and then the equations can be calculated in the topological order. This is the same as a spreadsheet would do.  
 2. If the model has **contemporaneous feedback** model is solved with an iterative method. Here variants of well known solution methods are used: 
     1. [Gauss-Seidle](https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method) (**Gauss**) which can handle large systems, is fairly robust and don't need the calculation of derivatives
     2. [Newthon-Raphson](https://en.wikipedia.org/wiki/Newton%27s_method) (**Newton**) which requires the calculation of derivatives and solving of a large linear system but typically converges in fewer iterations. 
 
Nearly all of the models solved by ModelFlow don't contain leaded endogenous variables. Therefor they can be solved one period at a time. For large sparse nonlinear models Gauss works fine. It solves a model quite fast and we don't need the additional handiwork of handling derivatives and large linear systems that Newton methods require. Moreover many models in question do not have smooth derivatives. The order in which the equation are calculated can have a large impact on the convergence speed. 

For some models the Newton algorithm works better. Some models are not able to converge with Gauss-Seidle other models are just faster using Newton. Also the ordering of equations does not matter for the convergence speed. 

However some models like FRB/US and other with **rational expectations** or **model consistent expectations** contains leaded endogenous variables. Such models typical has to be solved as one system for for all projection periods. In this case, the Gauss variation [Fair-Taylor](https://fairmodel.econ.yale.edu/rayfair/pdf/1983A.PDF) or Stacked-Newton Method.  The **stacked Newton** methods can be used in all cases, but if not needed, it will usually use more memory and be slower. 

| Model  | No contemporaneous feedback | Contemporaneous feedback | Leaded variables |
| --- | --- | --- |--- | 
|Normalized | Calculate | Gauss or <br> Newton  | Fair Taylor or <br> Stacked Newton|
|Un-normalized | Newton | Newton | Stacked Newton |

# Implementation of solving algorithms in Python
Solving a model entails a number of steps:

1. Specification of the model 
2. Create a dependency graph. 
2. Establish a solve order and separate the the model into smaller sub-models
2. Create a python function which can evaluating $f_i(y_1^{k},\cdots,y_{i-1}^{k},y_{i+1}^{k-1},\cdots,y_{n}^{k-1},z)$
3. If needed, create a python function which can evaluate the Jacobimatrices: $\bf{A,E,D}$  or $\bf{\bar A,\bar E,\bar D}$ 
3. Apply a solve function using the elements above to the data. 

 

### Normalized model


#### Calculation, No contemporaneous feedback

In systems with no lags each period can be solved in succession
The equations has to be evaluated in a logical (topological sorted) order. 

Let $z$ be all predetermined values: all exogenous variable and lagged endogenous variable.

Order the $n$ endogeneous variables in topological order.
    
For each time period we can find a solution by

for $i$ = 1 to $n$ 

>$y_{i}^{k} = f_i(y_1^{k},\cdots,y_{i-1}^{k},y_{i+1}^{k-1},\cdots,y_{n}^{k-1},z)$


#### The Gauss-Seidel algorithm. Normalized  models with contemporaneous feedback
The Gauss-Seidel algorithm is quite straight forward. It basically iterate over the formulas, until convergence. 

let:<br>
$z$ be all predetermined values: all exogenous variable and lagged endogenous variable.<br>
$n$ be the number of endogenous variables.<br>
$\alpha$ dampening factor which can be applyed to selected equations


   
For each time period we can find a solution by doing Gauss-Seidel iterations: 

for $k = 1$ to convergence

>for $i$ = 1 to $n$ 
>
>>$y_{i}^{k} = (1-\alpha) * y_{i}^{{k-1}} + \alpha f_i(y_1^{k},\cdots,y_{i-1}^{k},y_{i+1}^{k-1},\cdots,y_{n}^{k-1},z)$

#### The Newton-Raphson algorithme. Models with contemporaneous feedback

Let $\bf z$ be a vector all predetermined values: all exogenous variable and lagged endogenous variable.

For each time period we can find a solution by doing Newton-Raphson iterations:<br>

for $k = 1$ to convergence<br>

>$\bf y = \bf {F(y^{k-1},z) }$
>
>$\bf y^{k} =  \bf y - \alpha \times  \bf{(A-I)}^{-1} \times ( \bf {y - y^{k-1} })$

The expression: $\bf{(A-I)}^{-1}\times  ( \bf {y - y^{k-1} })$ is the same as the solution to: 

$\bf {y- y^{k-1} } = \bf (A-I) \times \bf x$

This problem can be solved much more efficient than performing $\bf{(A-I)}^{-1}\times  ( \bf {y - y^{k-1} })$

The Scipy library provides a number of solvers to this linear set of equations. There are both solvers using  factorization and iterative methods, and there are solvers for dense and sparce matrices. All linear solvers  can easily be incorporated into ModelFlows Newton-Raphson nonlinear solver.

#### Stacked Newton-Raphson all  periods in one go. Models with both leaded and lagged endogeneous variable

If the model has leaded endogenous variables it can in general not be solved one time period at a time. We have to solve the model for all time frames as one large model. 

$$\bf{\bar A} =\begin{bmatrix} 
		\bf{A_1}   & \bf{D_1^1} & \bf{D_1^2} & \bf{0}     &\bf{0}      &\bf{0}      &\bf{0}      &\bf{0}  \\
        \bf{E_2^1} & \bf{A_2}   & \bf{D_2^1} & \bf{D_2^2} &\bf{0}      &\bf{0}      &\bf{0}      &\bf{0} \\       
        \bf{E_3^2} & \bf{E_3^1} & \bf{A_3}   & \bf{D_3^1} & \bf{D_3^2} &\bf{0}      &\bf{0}      &\bf{0} \\       
        \bf{E_4^3} & \bf{E_4^2} & \bf{E_4^1} & \bf{A_4}   & \bf{D_4^1} & \bf{D_4^2} &\bf{0}      & \bf{0} \\       
        \bf{0}     & \bf{E_5^3} & \bf{E_5^2} & \bf{E_5^1} & \bf{A_5}   & \bf{D_5^1} & \bf{D_5^2} &\bf{0}\\       
        \bf{0}     & \bf{0}     & \bf{E_6^3} & \bf{E_6^2} & \bf{E_6^1} & \bf{A_6}   & \bf{D_6^1} & \bf{D_6^2}\\       
        \bf{0}     & \bf{0}     & \bf{0}     & \bf{E_7^3} & \bf{E_7^2} & \bf{E_7^1} & \bf{A_7}   & \bf{D_7^1} \\       
        \bf{0}     & \bf{0}     & \bf{0}     & \bf{0}     & \bf{E_8^3} & \bf{E_8^2} & \bf{E_8^1} & \bf{A_8} \\       
\end{bmatrix} \bar y = \begin{bmatrix}\bf{y_1}\\\bf{y_2}\\\bf{y_3}\\ \bf{y_4} \\\bf{y_5} \\\bf{y_6} \\ \bf{y_7} \\ \bf{y_8} \end{bmatrix} \bar F = \begin{bmatrix}\bf{F}\\\bf{F}\\\bf{F}\\ \bf{F} \\\bf{F} \\\bf{F} \\ \bf{F} \\ \bf{F} \end{bmatrix}$$


Now the solution algorithme looks like this. 

Again let $\bf z$ be a vector all predetermined values: all exogenous variable and lagged endogenous variable.

>for $k = 1$ to convergence<br>
>>$\bf{\bar y} = \bf {\bar F(\bar y^{k-1},\bar z) }$<br>
>>$\bf {\bar y^{k}} =  \bf{\bar y} - \alpha \times \bf{(\bar A-I)}^{-1}\times ( \bf {\bar y - \bar y^{k-1} })$

Notice that the model $\bf F$ is the same for all time periods.<br>
However, as time can be an exogenous variable the result of $\bf{F}$ can depend on time. This allows 
us to specify terminal conditions. 

The update frequency of $\bf{\bar A}$ and $\alpha$ and the value of $\alpha$ can be set to manage the speed and stability of the algorithm. 

We solve the problem: $$( \bf {\bar y - \bar y^{k-1} }) = \bf{(\bar A-I)}\times \bf x $$ instead of inverting  $\bf{A}$. 

Python gives access to very efficient sparse libraries. The [Scipy library](https://scipy.org/scipylib/index.html) utilizes the [Intel® Math Kernel Library](https://software.intel.com/en-us/mkl). Any of the available routines for solving linear systems can easily be incorporated. 

## Create a model instance which calculates the Jacobi matrices.
The derivatives of all formulas with respect to all endogenous variables are needed.    

First step is to specifying a model in the business logic language which calculate all the non-zero elements<br>
In ModelFlow this can be done by **symbolic**, by **numerical differentiation** or by a combination.  


The formula for calculating  $\dfrac{\partial{numerator }}{{\partial denominator(-lag)}}$ is written as:                             

\< numerator  \>\_\_p\_\_\< denominator >\_\_lag\_\_\< lag\> = derivative expression

Just another instance of a ModelFlow model class. 

### A small Solow model to show the construction of the Jacobi matrix. 
An example can be helpful<br> 
First a small model is defined - in this case a solow growth model: 

In [3]:
fsolow = '''\
Y         = a * k**alfa * l **(1-alfa) 
C         = (1-SAVING_RATIO)  * Y 
I         = Y - C 
diff(K)   = I-depreciates_rate * K(-1)
diff(l)   = labor_growth * L(-1) 
K_i= K/L '''
msolow = model.from_eq(fsolow)

In [4]:
print(msolow.equations)

FRML <> Y         = A * K**ALFA * L **(1-ALFA)  $
FRML <> C         = (1-SAVING_RATIO)  * Y  $
FRML <> I         = Y - C  $
FRML <> K=K(-1)+(I-DEPRECIATES_RATE * K(-1))$
FRML <> L=L(-1)+(LABOR_GROWTH * L(-1))$
FRML <> K_I= K/L  $


### Create some data and solve the model 

In [5]:
N = 100
df = pd.DataFrame({'L':[100]*N,'K':[100]*N})
df.loc[:,'ALFA'] = 0.5
df.loc[:,'A'] = 1.
df.loc[:,'DEPRECIATES_RATE'] = 0.05
df.loc[:,'LABOR_GROWTH'] = 0.01
df.loc[:,'SAVING_RATIO'] = 0.05
msolow(df,max_iterations=100,first_test=10,silent=1);

### Create an differentiation instance of the model
Use symbolic differentiation when possible else use numerical differentiation.  

In [6]:
from modelnewton import newton_diff
msolow.smpl(3,5);  # we only want a few years 

### Symbolic differentiation 

In [7]:
newton = newton_diff(msolow)
print(newton.diff_model.equations) 

* Problem sympify  K = K(-1)+(I-DEPRECIATES_RATE * K(-1))
* Problem sympify  L = L(-1)+(LABOR_GROWTH * L(-1))
FRML  <> C__p__Y___lag___0 = 1 - SAVING_RATIO   $
FRML  <> I__p__C___lag___0 = -1   $
FRML  <> I__p__Y___lag___0 = 1   $
FRML  <> K__p__I___lag___0 = 0   $
FRML  <> K__p__K___lag___1 = (((K(-1)+0.0025)+(I-DEPRECIATES_RATE*(K(-1)+0.0025)))-((K(-1)-0.0025)+(I-DEPRECIATES_RATE*(K(-1)-0.0025))))/0.005   $
FRML  <> K_I__p__K___lag___0 = 1/L   $
FRML  <> K_I__p__L___lag___0 = -K/L**2   $
FRML  <> L__p__L___lag___1 = (((L(-1)+0.0025)+(LABOR_GROWTH*(L(-1)+0.0025)))-((L(-1)-0.0025)+(LABOR_GROWTH*(L(-1)-0.0025))))/0.005   $
FRML  <> Y__p__K___lag___0 = A*ALFA*K**ALFA*L**(1 - ALFA)/K   $
FRML  <> Y__p__L___lag___0 = A*K**ALFA*L**(1 - ALFA)*(1 - ALFA)/L   $


### Numerical differentiation 

In [8]:
newton2 = newton_diff(msolow,forcenum=1)
print(newton2.diff_model.equations)

FRML  <> C__p__Y___lag___0 = (((1-SAVING_RATIO)*(Y+0.0025))-((1-SAVING_RATIO)*(Y-0.0025)))/0.005   $
FRML  <> I__p__C___lag___0 = ((Y-(C+0.0025))-(Y-(C-0.0025)))/0.005   $
FRML  <> I__p__Y___lag___0 = (((Y+0.0025)-C)-((Y-0.0025)-C))/0.005   $
FRML  <> K__p__I___lag___0 = ((K(-1)+((I+0.0025)-DEPRECIATES_RATE*K(-1)))-(K(-1)+((I-0.0025)-DEPRECIATES_RATE*K(-1))))/0.005   $
FRML  <> K__p__K___lag___1 = (((K(-1)+0.0025)+(I-DEPRECIATES_RATE*(K(-1)+0.0025)))-((K(-1)-0.0025)+(I-DEPRECIATES_RATE*(K(-1)-0.0025))))/0.005   $
FRML  <> K_I__p__K___lag___0 = (((K+0.0025)/L)-((K-0.0025)/L))/0.005   $
FRML  <> K_I__p__L___lag___0 = ((K/(L+0.0025))-(K/(L-0.0025)))/0.005   $
FRML  <> L__p__L___lag___1 = (((L(-1)+0.0025)+(LABOR_GROWTH*(L(-1)+0.0025)))-((L(-1)-0.0025)+(LABOR_GROWTH*(L(-1)-0.0025))))/0.005   $
FRML  <> Y__p__K___lag___0 = ((A*(K+0.0025)**ALFA*L**(1-ALFA))-(A*(K-0.0025)**ALFA*L**(1-ALFA)))/0.005   $
FRML  <> Y__p__L___lag___0 = ((A*K**ALFA*(L+0.0025)**(1-ALFA))-(A*K**ALFA*(L-0.0025)**(1-ALFA

### Display the full stacked matrix
To make the sparcity clear all zero values are shown as blank 


In [9]:
stacked_df = newton.get_diff_df_tot()
stacked_df.applymap(lambda x:f'{x:,.2f}' if x != 0.0 else ' ') 

per          3                                         4                       \
var          C      I      K    K_I      L      Y      C      I      K    K_I   
per var                                                                         
3   C    -1.00                               0.95                               
    I    -1.00  -1.00                        1.00                               
    K                  -1.00                                                    
    K_I                 0.01  -1.00  -0.01                                      
    L                                -1.00                                      
    Y                   0.51          0.49  -1.00                               
4   C                                              -1.00                        
    I                                              -1.00  -1.00                 
    K                   0.95                                     -1.00          
    K_I                                                           0.01  -1.00   
    L                                 1.01                                      
    Y                                                             0.51          
5   C                                                                           
    I                                                                           
    K                                                             0.95          
    K_I                                                                         
    L                                                                           
    Y                                                                           

per                        5                                     
var          L      Y      C      I      K    K_I      L      Y  
per var                                                          
3   C                                                            
    I                                                            
    K                                                            
    K_I                                                          
    L                                                            
    Y                                                            
4   C            0.95                                            
    I            1.00                                            
    K                                                            
    K_I  -0.01                                                   
    L    -1.00                                                   
    Y     0.49  -1.00                                            
5   C                  -1.00                               0.95  
    I                  -1.00  -1.00                        1.00  
    K                                -1.00                       
    K_I                               0.01  -1.00  -0.01         
    L     1.01                                     -1.00         
    Y                                 0.51          0.49  -1.00

### The results can also be displayed